In [42]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

import pandas as pd
import numpy as np
import json

from tqdm import tqdm

In [36]:
df = pd.read_csv('api_dataset.csv')
df.head(3)

,deadline,wantedTitle,jobId,workPlcNm,plDetAddr,etcItm,emplymShp,frDd,toDd,oranNm,stmNm,clltPrnnum,clerk,clerkContt,homepage
0,접수중,모라종합사회복지관 노인맞춤돌봄 생활지원사 채용 공고,K130232404150085,부산 사상구,"46934 부산광역시 사상구 모라로110번길 129 (모라동, 모라종합사회복지관)",장애인,기타,20240415,20240429,모라종합사회복지관,워크넷,3,이미정,051-304-9876,moraswc.or.kr
1,접수중,미화원 모집(밀레니엄빌딩),K151412404150070,경기 안산시 단원구,"15020 경기도 시흥시 함송로 15, 305호",장애인,시간제 일자리,20240415,20240430,삼성종합관리(주),워크넷,1,안상진,031-405-1168,시설관리.한국
2,접수중,아파트 설비기사 모집,K161232404150035,전북 익산시,"54555 전북특별자치도 익산시 무왕로25길 21 (부송동, 예린빌딩)","(준)고령자(50세이상), 고용촉진장려금대상자, 장애인",기타,20240415,20240520,(주)서영주택관리,워크넷,1,이창곤,063-843-2412,http://www.seoyoung.co.kr


### API 데이터셋에서 jobId만 추출

In [37]:
ids = df['jobId'].tolist()

## jobId를 활용해 워크넷 상세페이지 크롤링

In [26]:
import re

def text(elem):
    if not elem: 
        return 'nbsp'
    else: 
        text = []
        for txt in elem:
            text.append(txt.text)
        return text
        
def preprocess(txt):
    clean_text = re.sub(r'[\n\t\r]+', ' ', ' '.join(txt))
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    
    return clean_text
    

In [46]:
final = [] 

for idnum in ids[:10]:
    
    try: 
        data = {'jobId' :idnum}
        url = f'https://www.work.go.kr/empInfo/empInfoSrch/detail/empDetailAuthView.do?searchInfoType=VALIDATION&callPage=detail&wantedAuthNo={idnum}&rtnUrl=/empInfo/empInfoSrch/list/dtlEmpSrchList.do'

        response = requests.get(url)
        response.raise_for_status() 
        soup = BeautifulSoup(response.content, 'html.parser')

        #직무 내용
        job_desc = soup.select('div.careers-area > div:nth-child(4) > table > tbody > tr > td')

        #경력 조건
        career = soup.select('div:nth-child(5) > table > tbody > tr > td:nth-child(1)') #경력 조건
        edu = soup.select('div:nth-child(5) > table > tbody > tr > td:nth-child(2)') #학력
        employform = soup.select('div:nth-child(5) > table > tbody > tr > td:nth-child(3)') #고용형태
        disabled = soup.select('div:nth-child(5) > table > tbody > tr > td:nth-child(5)') #장애인 채용인원
        workplace = soup.select('div:nth-child(5) > table > tbody > tr > td:nth-child(6)') #근무예정지

        #모집 직종 (직종 키워드)
        worktype = soup.select('div:nth-child(6) > table > tbody > tr > td:nth-child(1)') #모집 직종
        keyword = soup.select('div:nth-child(6) > table > tbody > tr > td:nth-child(2)') #직종 키워드 
        related = soup.select('div:nth-child(6) > table > tbody > tr > td:nth-child(3)') #관련 직종

        #근무조건
        pay = soup.select('div:nth-child(8) > table > tbody > tr > td:nth-child(1)') #임금
        workinghours = soup.select('div:nth-child(8) > table > tbody > tr > td:nth-child(2)') #근무시간
        workform = soup.select('div:nth-child(8) > table > tbody > tr > td:nth-child(3)') #근무형태
        ensurance = soup.select('div:nth-child(8) > table > tbody > tr > td:nth-child(4)') #사회보험
        retirement = soup.select('div:nth-child(8) > table > tbody > tr > td:nth-child(5)') #퇴직급여

        #전형 방법 
        method = soup.select('div:nth-child(11) > table > tbody > tr > td:nth-child(2)') #전형, 방법
        apply = soup.select('div:nth-child(11) > table > tbody > tr > td:nth-child(3)') #접수 방법
        prepare = soup.select('div:nth-child(11) > table > tbody > tr > td:nth-child(4)') #제출서류 준비물

        #우대조건
        major = soup.select('div:nth-child(14) > table > tbody > tr > td:nth-child(1)')
        certificate = soup.select('div:nth-child(14) > table > tbody > tr > td:nth-child(2)') #자격면허
        foreign = soup.select('div:nth-child(14) > table > tbody > tr > td:nth-child(3)') #외국어 능력
        army = soup.select('div:nth-child(14) > table > tbody > tr > td:nth-child(4)') #병역대체 복무자 채용
        accept = soup.select('div:nth-child(14) > table > tbody > tr > td:nth-child(5)') #고용허가제
        first = soup.select('div.careers-table.cnts.v1.mt20 > table > tbody > tr:nth-child(2) > td:nth-child(1)') #우대조건
        computer = soup.select('div.careers-table.cnts.v1.mt20 > table > tbody > tr:nth-child(2) > td:nth-child(2)') #컴퓨터 활용능력
        envir = soup.select('div.careers-table.cnts.v1.mt20 > table > tbody > tr:nth-child(2) > td:nth-child(2)') #작업환경

        #기타사항
        etc = soup.select('div:nth-child(17) > table > tbody > tr > td')

        #복리후생
        welfare = soup.select('div.careers-welfare-list > ul > li.on > p')


        #상세 정보 정의
        categories = {
            '직무 내용' : job_desc,'경력 조건' : career,
            '학력' : edu,'고용형태': employform,
            '장애인 채용' : disabled,'근무지' : workplace,
            '모집 직종' : worktype,'직종 키워드' : keyword,
            '관련 직종' : related,'임금' : pay,
            '근무시간' : workinghours ,'근무형태' : workform,
            '사회보험' : ensurance,'퇴직급여' : retirement,
            '전형/방법' : method,'접수 방법' : apply,
            '제출서류' : prepare,'전공 우대' : major,
            '자격면허' : certificate,'외국어 능력' : foreign,
            '병역대체 복무자' : army, '고용허가제' : accept,
            '우대조건' : first,'컴퓨터 활용능력' : computer,
            '작업환경' : envir,'기타사항' : etc,
            '복리후생' : welfare
        }


        for category, elements in categories.items():
            elements = text(elements)
            data[category] = preprocess(elements)

        df = pd.DataFrame({k: [v] for k, v in data.items()})
       
        
    except requests.exceptions.RequestException as e:
        data = {'jobId' :idnum}
        for category in categories:
            data[category] = np.nan
        df = pd.DataFrame([data])
    
    final.append(df)
        

#데이터프레임 결합
specific_info = pd.concat(final,ignore_index=True)
specific_info.replace('n b s p', np.nan, inplace=True)

In [48]:
specific_info

,jobId,직무 내용,경력 조건,학력,고용형태,장애인 채용,근무지,모집 직종,직종 키워드,관련 직종,...,전공 우대,자격면허,외국어 능력,병역대체 복무자,고용허가제,우대조건,컴퓨터 활용능력,작업환경,기타사항,복리후생
0,K130232404150085,"- 활동권역: 모라1,3동 - 직접 서비스 제공(가사,정서,안부 서비스 등), 서비...",관계없음,학력무관,기간의 정함이 있는 근로계약 8개월,1명 (장애인 병행채용),"부산광역시 사상구 모라로110번길 129 (모라동, 모라종합사회복지관)",노인 생활지도원,"노인생활지도원 , 노인생활복지사","사회복지사, 재가 요양보호사",...,,관계없음,,비희망,,"장애인 ,",,,,NaN
1,K151412404150070,"건물청소 월급 1,789,590원 (연차수당 포함, 퇴직금 별도)",경력 (최소 1년 0 개월 이상) 우대,학력무관,기간의 정함이 있는 근로계약(시간(선택)제) 3개월/ 계약기간 만료 후 상용직전환검토,1명 (장애인 병행채용),"경기도 안산시 단원구 광덕대로 141, 밀레니엄빌딩 (고잔동)","건물 청소원(공공건물,아파트,사무실,병원,상가,공장 등)","상가청소원, 병원청소원, 사우나청소원",,...,,관계없음,,비희망,,"장애인 ,",,,,NaN
2,K161232404150035,공용시설물 관리,경력 (최소 1년 0 개월 이상) 필수,학력무관,기간의 정함이 있는 근로계약 12개월,1명 (장애인 병행채용),"전북특별자치도 익산시 선화로53길 10, 아파트 관리사무소 (마동, 익산 오투그란데...",건물 보수원 및 영선원(아파트 기계·전기 시설관리 제외),"건물영선원, 공장보수원 , 건물수리원, 공장영선원 , 건물보수원, 공장시설관리원 ,...",,...,,2급 소방안전관리자(필수),,비희망,,"장애인 ,(준)고령자(50세이상) ,고용촉진장려금대상자",,,,NaN
3,K151632404150063,**회사정보 : www.idlabs.co.kr 마케터 를 채용합니다. #아이디랩스의...,경력 (최소 1년 0 개월 이상) 우대,학력무관,기간의 정함이 없는 근로계약,1명 (장애인 병행채용),"경기도 성남시 수정구 고등로 3, A231호 (고등동, 현대지식산업센터 성남고등)",웹 기획자,,마케팅·광고·홍보·상품기획 사무원,...,,관계없음,,비희망,,"장애인 ,",,,,식사제공 교육비지원
4,K120612404150051,파로스프라자 상가아파트 미화원 모집 평일 3시간 근무 : 09:00 ~ 12:00,관계없음,학력무관,기간의 정함이 있는 근로계약 2개월,1명 (장애인 병행채용),"서울특별시 관악구 난곡로 100, 상가미화 (신림동, 파로스프라자A동)","건물 청소원(공공건물,아파트,사무실,병원,상가,공장 등)","아파트청소원, 사무실청소원, 병원청소원, 역구내정리원, 매장청소원",,...,,관계없음,,비희망,,"장애인 ,(준)고령자(50세이상) ,고용촉진장려금대상자",,,,NaN
5,KJAT002404150003,- 목동현대 아파트 미화청소 / 복지카드 우대 - 서울시 양천구 목동동로 12길 6...,관계없음,학력무관,기간의 정함이 있는 근로계약 2개월/ 계약기간 만료 후 상용직전환검토,1명 (장애인 병행채용),"서울특별시 양천구 목동동로12길 60, 0 (신정동, 목동현대아파트)","건물 청소원(공공건물,아파트,사무실,병원,상가,공장 등)",,청소원,...,,관계없음,,비희망,,"장애인 ,(준)고령자(50세이상) ,",,,,NaN
6,K120422404150054,공통 자격 요건 ㆍ학력 : 고졸이상 해외 송금 서비스 인바운드 CS 상담 마케팅 담...,관계없음,고졸 ~ 대졸(4년),기간의 정함이 없는 근로계약,2명 (장애인 병행채용),"서울특별시 영등포구 영등포로 150, 901호~911호 (당산동1가, 생각공장 당산)",콜센터 상담원(콜센터·고객센터·CS센터),"콜센터상담원 , 인바운드 텔레마케터",,...,,관계없음,,비희망,,"장애인 ,고용촉진장려금대상자,보훈취업지원대상자",,,,식사제공
7,K151632404150065,단지 내 아파트 외곽 청소 월-금 : (오전) 8시 30분~(오후) 3시 30분 토...,관계없음,학력무관,기간의 정함이 있는 근로계약 12개월,1명 (장애인 병행채용),"경기도 수원시 팔달구 화양로50번길 30, 화서역블루밍 푸른숲아파트 (화서동, 화서...","건물 청소원(공공건물,아파트,사무실,병원,상가,공장 등)","아파트청소원, 사무실청소원, 병원청소원, 역구내정리원, 매장청소원",,...,,관계없음,,비희망,,"장애인 ,",,,본사를 거치지 않고 임의로 관리사무소 방문 시 미채용됨을 알려드립니다.,NaN
8,K161222404150009,누룽지 생산 현장에서 오븐에 밥 짓기 및 기계에 밥 투입.,관계없음,학력무관,기간의 정함이 없는 근로계약,1명 (장애인 병행채용),"전북특별자치도 김제시 금구면 양시3길 107-38, 박씨네누룽지영농조합법인",식품 분야 단순 종사원,식품분야 단순노무자,,...,,관계없음,,비희망,,"장애인 ,(준)고령자(50세이상) ,고용촉진장려금대상자",,,,통근버스 식사제공
9,K120612404150060,아파트 경비 업무 / 격일제 교대 근무 / 휴게시간 ( 주간 3 야간 7 총 10 ...,관계없음,학력무관,기간의 정함이 있는 근로계약 2개월,1명 (장애인 병행채용),"서울특별시 동작구 국사봉길 178, 경비 (상도동, 상도1차 갑을명가아파트)",아파트·빌라 경비원,아파트경비원,경비원(건물 관리원),...,NaN,NaN,NaN,NaN,NaN,"장애인 ,(준)고령자(50세이상) ,고용촉진장려금대상자",,,NaN,NaN
